In [98]:
import pandas as pd
import multiprocessing
import tqdm
import pickle
import re

from mypostgres import PostgresConnection

from scraper import carProperties

from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

import getpass


In [107]:
from importlib import reload
import mypostgres as pgs

## Scrape Pages in parallel

In [2]:
cars_list = []

def get_car_data(url):
    return carProperties(url).get_data()

with open('/home/leon/Documents/repos/car-prices-analysis/crawler/links copy.csv','r') as f:
    
    urls = [l.strip('\n') for l in f.readlines()]
    cars_list = Parallel(n_jobs=num_cores)(delayed(get_car_data)(url) for url in tqdm.tqdm(urls))

    cars_df = pd.concat(cars_list,axis=1).T

100%|██████████| 760/760 [05:06<00:00,  2.48it/s]


In [23]:
cars_df = cars_df.infer_objects()
cars_df['power'] = cars_df['power'].astype('int')
cars_df[['brand','model']] = cars_df[['brand','model']].astype('category')

In [39]:
with open('cars_data.pkl','wb') as f:
    pickle.dump(cars_df,f)
    

In [108]:
with pgs.PostgresConnection(dbname='random',host='192.168.1.218',schema='cars',user='leon',password=getpass.getpass('PSQL password: ')) as conn:
    conn.create_table_from_df(cars_df,'fct_cars')

TypeError: __init__() got an unexpected keyword argument 'schema'

## Run regression

In [94]:
mod = smf.ols(formula='price ~ 0 + C(brand) + C(model) + ' + str(' + '.join(all_cols)), data=cars_df)
res = mod.fit()
print(res.summary())

OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     70.82
Date:                Sun, 10 May 2020   Prob (F-statistic):          9.82e-286
Time:                        18:38:21   Log-Likelihood:                -6496.2
No. Observations:                 752   AIC:                         1.318e+04
Df Residuals:                     659   BIC:                         1.361e+04
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
C(brand)[Audi]                    53